# Payload Configuration Sweep
Similar to the spreadsheet from before.

The inputs to the sweep are the following:
* Payload configurations to try
* Assumed parameters (e.g. L/D, target speeds)
* Historical data (for aerostructural mass ratio)

We could pick different historical data to infer the AS ratio from. For example, we could do all aircraft instead of just Rassam's picks.

The code also supports swapping in custom sets of components for e.g. if our mass budget changes.


In [4]:
import pandas as pd
from IPython.display import display

from wyvern.data import RASSAM_CORRELATIONS, ALL_COMPONENTS
from wyvern.analysis.payload_sweep import sweep_payload_configs
from wyvern.analysis.parameters import PayloadSizingParameters
from wyvern.sizing import aerostructural_mass_ratio, total_component_mass


# calculate aerostructural mass ratio and fixed component masses first
total_fixed_mass = total_component_mass(ALL_COMPONENTS)
as_m_ratio = aerostructural_mass_ratio(RASSAM_CORRELATIONS, total_fixed_mass)

params = PayloadSizingParameters(
    as_mass_ratio=as_m_ratio,
    total_fixed_mass=total_fixed_mass,
    lift_to_drag_ratio=10,
    cruise_speed=12,
    turn_speed=11,
    propulsive_efficiency=0.5,
    configuration_bonus=1.3,
    short_takeoff=False,
    stability_distance=0,
)

payload_configs = [(8, i, 4) for i in range(0, 7)]
results = sweep_payload_configs(payload_configs, params)

with pd.option_context("display.precision", 3):
    display(results)

num_ping_pong_balls num_golf_balls num_tennis_balls payload_mass  \
8 0 4                   8              0                4        259.2   
  1 4                   8              1                4       305.13   
  2 4                   8              2                4       351.06   
  3 4                   8              3                4       396.99   
  4 4                   8              4                4       442.92   
  5 4                   8              5                4       488.85   
  6 4                   8              6                4       534.78   

      empty_mass  as_mass takeoff_mass cargo_units payload_fraction  \
8 0 4    1033.06   584.91      1292.26         480            0.201   
  1 4    1071.04   622.89      1376.17         530            0.222   
  2 4   1109.019  660.869     1460.079         580             0.24   
  3 4   1146.999  698.849     1543.989         630            0.257   
  4 4   1184.979  736.829     1627.899         680            0.272   
  5 4   1222.958  774.808     1711.808         730            0.286   
  6 4   1260.938  812.788     1795.718         780            0.298   

      reached_pf_cap cargo_score_times_pf total_energy efficiency_score  \
8 0 4          False               15.106458     2456.317            1.697   
  1 4          False               17.898     2615.812            1.497   
  2 4          False               20.673     2775.306            1.329   
  3 4           True               22.776     2934.801            1.189   
  4 4           True               24.027     3094.296            1.069   
  5 4           True                25.25     3253.791            0.967   
  6 4           True               26.449     3413.285            0.879   

      takeoff_bonus configuration_bonus stability_bonus total_flight_score  
8 0 4           1.0                 1.3             1.0             33.329  
  1 4           1.0                 1.3             1.0             34.821  
  2 4           1.0                 1.3             1.0             35.729  
  3 4           1.0                 1.3             1.0             35.202  
  4 4           1.0                 1.3             1.0             33.405  
  5 4           1.0                 1.3             1.0             31.749  
  6 4           1.0                 1.3             1.0             30.221

## Additional Math

### Energy
Assumptions:
* Constant L/D
* Constant speed steady level flight and level turn, with no transition time between the two

In steady level flight,
$$ L = W $$
$$ D = \frac{L}{L/D} $$
$$ T = D$$
$$ P = T v = \frac{W}{L/D} v $$
$$ E_{\text{SLF}} = \frac{W}{L/D} v_{\text{SLF}} t_{\text{SLF}} $$

In a turn, the lift is increased by the load factor. The load factor is given by:
$$ n = \sqrt{\left(\frac{v^2}{g R}\right)^2 + 1}$$
$$ L = W n $$
and similarly as before,
$$ E_{\text{SLT}} = \frac{nW}{L/D} v_{\text{SLT}} t_{\text{SLT}} $$

To course is split into straights of length $\ell_s$ and turns of length $\ell_t$. The time spent in each is given by:
$$ t_{\text{SLF}} = \frac{\ell_s}{v_{\text{SLF}}} $$
$$ t_{\text{SLT}} = \frac{\ell_t}{v_{\text{SLT}}} $$

and the total energy is given by:
$$ E = \frac{W}{L/D} v_{\text{SLF}} \frac{\ell_s}{v_{\text{SLF}}} + \frac{nW}{L/D} v_{\text{SLT}} \frac{\ell_t}{v_{\text{SLT}}} $$
$$ E = \frac{W}{L/D} \ell_s + \frac{nW}{L/D} \ell_t $$
$$ E = \frac{W}{L/D} \left(\ell_s + n \ell_t\right)$$

and we will put the propulsive efficiency $\eta$ in the denominator:
$$ \boxed{E = \frac{W}{\eta L/D} \left(\ell_s + n \ell_t\right)} $$

which gives an underestimate of the energy required. The actual energy will be greater due to transients and other losses. Purportedly, 3300 J is a "very good" performance.